Problemas detectados:

* Se está usando el metadato del título de Google Scholar. El metadato de título debería ser el de la mejor calidad posible 
* Revisar el uso de los autores en la creación del link de Google Lookup

In [1]:
import pandas as pd                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [2]:
df=pd.read_csv('data/1-Producci_n_Grupos_Investigaci_n.csv',low_memory=False)#,nrows=100)

In [3]:
df.shape

(2254793, 14)

In [4]:
df=df[df['NME_TIPOLOGIA_PD']=='Artículos de investigación'].reset_index(drop=True)

In [5]:
df=df.sample(1000).reset_index(drop=True)

In [23]:
df.to_csv('data/open_scienti_sample.csv',index=False)

In [7]:
co=pd.read_json('data/scholar_colombia.json',orient='records',lines=True)#,nrows=1)

In [8]:
co=co.drop(['abstract','cites_link','lookup_url'],axis='columns')

In [9]:
co.shape

(181040, 21)

In [10]:
import wosplus as wp

In [11]:
wp.merge_with_close_matches?

Signature:
wp.merge_with_close_matches(
    left,
    right,
    left_on='ST',
    right_on='UDEA_simple_título',
    left_extra_on='SO',
    right_extra_on='UDEA_nombre revista o premio',
    how='inner',
    n=1,
    cutoff=0.6,
    full=True,
    cutoff_extra=0.6,
)
Docstring:
For each entry of the column: left_on of DataFrame left (cannot have empty fields),
try to find the close match inside each row of right DataFrame, by comparing with
the right_on entry of the row. When a row match is found, the full right row is appended
to the matched row in the left DataFrame.
If the similarity between the entries at left_on and right_on is less than 0.8,
an additional check is performed between the entries left_extra_on and right_extra_on
of the matched row.

how implemented: inner and left (Default: inner)
File:      /usr/local/lib/python3.9/dist-packages/wosplus/_wos_scp.py
Type:      function


In [12]:
df.columns

Index(['ID_CONVOCATORIA', 'NME_CONVOCATORIA', 'ANO_CONVO', 'ID_PRODUCTO_PD',
       'NME_CLASE_PD', 'NME_TIPO_MEDICION_PD', 'NME_TIPOLOGIA_PD',
       'ID_TIPO_PD_MED', 'NME_CATEGORIA_PD', 'FCREACION_PD', 'NME_PRODUCTO_PD',
       'COD_GRUPO_GR', 'NME_GRUPO_GR', 'ID_PERSONA_PD'],
      dtype='object')

In [18]:
df['year']=df['FCREACION_PD'].str.split('/').str[-1]

In [13]:
co.columns

Index(['_id', 'title', 'author', 'profiles', 'cites', 'journal', 'publisher',
       'country', 'doi', 'year', 'volume', 'issue', 'pages', 'ref', 'pdf',
       'cid', 'rp', 'bibtex', 'old_title', 'article_id', 'timestamp'],
      dtype='object')

In [19]:
mt=wp.merge_with_close_matches(df[['NME_PRODUCTO_PD','year']],co[['title', 'year']],
                              left_on='NME_PRODUCTO_PD',
                              right_on='title',
                              left_extra_on='year',
                              right_extra_on='year',
                              how='left',                              
                              )

..........

In [25]:
mt=mt.dropna(subset=['title'])

In [29]:
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows', 1000)

In [39]:
from fuzzywuzzy import fuzz
from unidecode import unidecode

In [40]:
unidecode('á')

'a'

In [53]:
import re

In [122]:
mt['ratio']=mt.apply(lambda row:  fuzz.ratio( re.sub(r'\\\w+\s*',r'',
                                              unidecode(row['NME_PRODUCTO_PD'].lower().strip(
                                              ).replace('.','').replace('$','') ) 
                                                    ),
                                              re.sub(r'\\\w+\s*',r'',
                                              unidecode(row['title'].lower().strip(
                                              ).replace('.','').replace('$','') )
                                                    )
                                                    ),
                                 axis='columns')

/tmp/ipykernel_132733/2140895428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt['ratio']=mt.apply(lambda row:  fuzz.ratio( re.sub(r'\\\w+\s*',r'',


In [123]:
mt[ (mt['ratio']>80) ].shape #& (mt['ratio']<90) ].reset_index(drop=True)

(535, 5)

In [130]:
co['Gtitle']=co['bibtex'].str.split('title=\{').str[-1].str.split('\},\\n').str[0]

In [133]:
mt=mt.merge(co[['title','Gtitle']],on='title',how='left')

In [134]:
mt[ (mt['ratio']>80) & (mt['ratio']<90) ].reset_index(drop=True)

,NME_PRODUCTO_PD,year,title,year,ratio,Gtitle
0,"Riqueza genérica y distribución de Elmidae (Insecta: Coleoptera, Byrrhoidea) en el departamento del Valle del Cauca, Colombia",2015,"Riqueza y distribución de Elmidae (Insecta: Coleoptera: Byrrhoidea) en el departamento del Chocó, Colombia",2016,88,"Riqueza y distribuci{\'o}n de Elmidae (Insecta: Coleoptera: Byrrhoidea) en el departamento del Choc{\'o}, Colombia"
1,Efficient Software Implementation of the Nearly Optimal Sparse Fast Fourier Transform for the Noisy Case,2015,"Efficient Software Implementation of the Nearly Optimal Sparse Fast Fourier Transform for the Noisy Case. ing. cienc.[online]. 2015, vol. 11, n. 22",1794,85,"Efficient Software Implementation of the Nearly Optimal Sparse Fast Fourier Transform for the Noisy Case. ing. cienc.[online]. 2015, vol. 11, n. 22"
2,Gastric mucosa-Associated lymphoid tissue lymphomas and Helicobacter pylori infection,2012,Gastric mucosa-associated lymphoid tissue lymphomas and Helicobacter pylori infection: a Colombian perspective,2012,87,Gastric mucosa-associated lymphoid tissue lymphomas and Helicobacter pylori infection: a Colombian perspective
3,Determinación de anticuerpos totales (IgG/IgM) y específicos (IgM) para el virus de la hepatitis E y detección molecular del virus en heces de humanos con o sin exposición ocupacional a porcinos e...,2015,Determinação de anticorpos totais (IgG/IgM) e especı́ficos (IgM) para o v\ŕus da hepatite E e detecção molecular do v\'ús em fezes de humanos com ou sem exposição ocupacional a porcinos em 10 muni...,2015,86,Determina{\c{c}}{\~a}o de anticorpos totais (IgG/IgM) e espec{\'\i}ficos (IgM) para o v{\'\i}rus da hepatite E e detec{\c{c}}{\~a}o molecular do v{\'\i}rus em fezes de humanos com ou sem exposi{\c...
4,Measurements of inclusive W and Z cross sections in pp collisions at &#8730;s = 7 TeV The CMS collaboration,2011,Measurements of inclusive W and Z cross sections in pp collisions at $$\backslash$sqrt $\$s$\$= 7$ TeV,2011,83,Measurements of inclusive W and Z cross sections in pp collisions at $$\backslash$sqrt $\{$s$\}$= 7$ TeV
5,112. Hydrostatic pressure and electric and magnetic field effects on the binding energy of a hydrogenic donor impurity in InAs Poschl-Teller quantum ring,2012,"Hydrostatic pressure, impurity position and electric and magnetic field effects on the binding energy and photo-ionization cross section of a hydrogenic donor impurity in an InAs Pöschl-Teller qua...",2011,83,"Hydrostatic pressure, impurity position and electric and magnetic field effects on the binding energy and photo-ionization cross section of a hydrogenic donor impurity in an InAs P{\""o}schl-Teller..."
6,Crítica al control jerárquico de los regímenes políticos: complejidad y topología,2015,Crı́tica ao controle hierárquico dos regimes pol\t́icos: complexidade e topologia,2015,86,Cr{\'\i}tica ao controle hier{\'a}rquico dos regimes pol{\'\i}ticos: complexidade e topologia
7,The interplay between sharing behavior and beliefs about others in children.,2018,The interplay between sharing behavior and beliefs about others in children during dictator games,2018,87,The interplay between sharing behavior and beliefs about others in children during dictator games
8,"La estructura de capital en las medianas empresas del departamento de Boyacá, Colombia",2015,"La estructura de capital en las medianas empresas del departamento de Boyacá, Colombia. Apuntes del CENES, 34 (59), 185-206",2015,83,"La estructura de capital en las medianas empresas del departamento de Boyac{\'a}, Colombia. Apuntes del CENES, 34 (59), 185-206"
9,Letters to editor Laryngeal histoplasmosis: report first case in Colombia,2015,Laryngeal Histoplasmosis: report first case in Colombia,2014,86,Laryngeal Histoplasmosis: report first case in Colombia
